<a href="https://colab.research.google.com/github/amandinha-gomes/analise_dos_gastos_da_CEAPS_Senado_Federal/blob/main/analise_CEAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Leitura e tratamento

In [11]:
df = pd.read_csv(
    "despesa_ceaps_2022.csv",
    sep=";",
    encoding="latin-1",
    skiprows=1
)

In [12]:
print(df.columns)
df.head()

Index(['ANO', 'MES', 'SENADOR', 'TIPO_DESPESA', 'CNPJ_CPF', 'FORNECEDOR',
       'DOCUMENTO', 'DATA', 'DETALHAMENTO', 'VALOR_REEMBOLSADO',
       'COD_DOCUMENTO'],
      dtype='object')


,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,18/01/2022,Divulgação da atividade parlamentar,1000,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,17/01/2022,Divulgação da atividade parlamentar,2000,2173617


In [14]:
df["VALOR_REEMBOLSADO"] = (
    df["VALOR_REEMBOLSADO"]
    .astype(str)
    .str.replace(",", ".")
    .astype(float)
)

### Estatísticas macro do dataset

In [16]:
total_despesas = df.shape[0]
total_senadores = df["SENADOR"].nunique()
valor_total = df["VALOR_REEMBOLSADO"].sum()
media_por_senador = valor_total / total_senadores

print(f"Total de despesas: {total_despesas}")
print(f"Total de senadores: {total_senadores}")
print(f"Valor total reembolsado: R$ {valor_total:,.2f}")
print(f"Média por senador: R$ {media_por_senador:,.2f}")

Total de despesas: 16805
Total de senadores: 97
Valor total reembolsado: R$ 27,323,316.33
Média por senador: R$ 281,683.67


### Ranking — quem gastou mais?

In [17]:
gastos_senador = (
    df.groupby("SENADOR", as_index=False)["VALOR_REEMBOLSADO"]
      .sum()
      .sort_values("VALOR_REEMBOLSADO", ascending=False)
)

gastos_senador.head(10)

,SENADOR,VALOR_REEMBOLSADO
50,LUCAS BARRETO,511319.78
14,DAVI ALCOLUMBRE,505054.54
68,OMAR AZIZ,494369.33
90,TELMÁRIO MOTA,488693.40
64,MECIAS DE JESUS,488586.66
81,ROGÉRIO CARVALHO,487764.65
75,RANDOLFE RODRIGUES,487758.92
9,CHICO RODRIGUES,486958.05
54,MAILZA GOMES,465899.61
23,ELMANO FÉRRER,465194.93


#### Gráfico – Top 10 senadores

In [18]:
fig = px.bar(
    gastos_senador.head(10),
    x="VALOR_REEMBOLSADO",
    y="SENADOR",
    orientation="h",
    title="Top 10 Senadores com Maior Gasto em 2022",
    labels={
        "VALOR_REEMBOLSADO": "Valor Reembolsado (R$)",
        "SENADOR": ""
    }
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

### Distribuição dos gastos (outliers)

In [19]:
fig = px.box(
    gastos_senador,
    y="VALOR_REEMBOLSADO",
    title="Distribuição dos Gastos Totais por Senador"
)

fig.show()

### Gastos ao longo do ano (sazonalidade)

In [20]:
gastos_mes = (
    df.groupby("MES", as_index=False)["VALOR_REEMBOLSADO"]
      .sum()
)

fig = px.line(
    gastos_mes,
    x="MES",
    y="VALOR_REEMBOLSADO",
    markers=True,
    title="Evolução Mensal dos Gastos da CEAPS em 2022",
    labels={
        "MES": "Mês",
        "VALOR_REEMBOLSADO": "Valor Reembolsado (R$)"
    }
)

fig.show()

### Tipos de despesa — para onde vai o dinheiro?

In [21]:
gastos_tipo = (
    df.groupby("TIPO_DESPESA", as_index=False)["VALOR_REEMBOLSADO"]
      .sum()
      .sort_values("VALOR_REEMBOLSADO", ascending=False)
)

gastos_tipo.head(10)

,TIPO_DESPESA,VALOR_REEMBOLSADO
5,"Passagens aéreas, aquáticas e terrestres nacio...",7423646.13
2,"Contratação de consultorias, assessorias, pesq...",6321484.03
4,"Locomoção, hospedagem, alimentação, combustíve...",5259803.51
0,"Aluguel de imóveis para escritório político, c...",3976536.92
3,Divulgação da atividade parlamentar,3345915.36
1,Aquisição de material de consumo para uso no e...,983707.82
6,Serviços de Segurança Privada,12222.56


In [22]:
fig = px.bar(
    gastos_tipo.head(8),
    x="VALOR_REEMBOLSADO",
    y="TIPO_DESPESA",
    orientation="h",
    title="Principais Tipos de Despesa da CEAPS (2022)",
    labels={
        "VALOR_REEMBOLSADO": "Valor Reembolsado (R$)",
        "TIPO_DESPESA": ""
    }
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

### Tipo de despesa por mês (eleições)

In [23]:
gastos_mes_tipo = (
    df.groupby(["MES", "TIPO_DESPESA"], as_index=False)["VALOR_REEMBOLSADO"]
      .sum()
)

fig = px.bar(
    gastos_mes_tipo,
    x="MES",
    y="VALOR_REEMBOLSADO",
    color="TIPO_DESPESA",
    title="Gastos Mensais por Tipo de Despesa – 2022",
)

fig.show()

### Senadores que mais concentraram gastos

In [24]:
top5 = gastos_senador.head(5)["SENADOR"]

df_top5 = df[df["SENADOR"].isin(top5)]

fig = px.line(
    df_top5.groupby(["MES", "SENADOR"], as_index=False)["VALOR_REEMBOLSADO"].sum(),
    x="MES",
    y="VALOR_REEMBOLSADO",
    color="SENADOR",
    markers=True,
    title="Evolução Mensal dos Gastos – Top 5 Senadores"
)

fig.show()